<a href="https://colab.research.google.com/github/nd20104/CE888/blob/main/Assignment-2/tweetClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import important libraries

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import csv
import matplotlib.pyplot as plt


In [ ]:
import seaborn as sns
import re
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import  CountVectorizer, TfidfVectorizer
from sklearn import svm



In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud


My chosen datasets are: **Offensive language identification, Hate speech detection and Emotion recongnition**

## Loading *Emotion datasets*

In [ ]:

#traindf=pd.read_csv('../input/semeval-2018-task-ec/2018-E-c-En-train.txt',encoding='utf-8',sep="\t")
emo_train_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_text.txt",encoding='utf-8',sep="\t", header=None)
emo_train_text.columns= ["tweet_text"]
emo_train_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/train_labels.txt",encoding='utf-8',sep="\t", header=None)
emo_train_label.columns= ["tweet_label"]


emo_val_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_text.txt",encoding='utf-8',sep="\t", header=None)
emo_val_text.columns= ["tweet_text"]
emo_val_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_labels.txt",encoding='utf-8',sep="\t", header=None)
emo_val_label.columns= ["tweet_label"]

emo_test_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt",encoding='utf-8',sep="\t", header=None)
emo_test_text.columns= ["tweet_text"]
emo_test_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_labels.txt",encoding='utf-8',sep="\t", header=None)
emo_test_label.columns= ["tweet_label"]

emo_map = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/mapping.txt",encoding='utf-8',sep="\t", header=None)
emo_map.columns= ["tweet_label","emotion"]


## Loading *Hate datasets*

In [ ]:
#loading Hate speech data
hate_train_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_text.txt", sep="\t", quoting=csv.QUOTE_NONE, skip_blank_lines=False, header=None)
hate_train_text.columns= ["tweet_text"]
hate_train_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_labels.txt",encoding='utf-8',sep="\n", header=None)
hate_train_label.columns= ["tweet_label"]


hate_val_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_text.txt",sep="\t", quoting=csv.QUOTE_NONE, skip_blank_lines=False, header=None)
hate_val_text.columns= ["tweet_text"]
hate_val_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_labels.txt",encoding='utf-8',sep="\t", header=None)
hate_val_label.columns= ["tweet_label"]

hate_test_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt",sep="\t", quoting=csv.QUOTE_NONE, skip_blank_lines=False, header=None)
hate_test_text.columns= ["tweet_text"]
hate_test_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_labels.txt",encoding='utf-8',sep="\t", header=None)
hate_test_label.columns= ["tweet_label"]

hate_map = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/mapping.txt",encoding='utf-8',sep="\t", header=None)
hate_map.columns= ["tweet_label","isHate"]

## Loading *Offensive datasets*

In [ ]:
#loading offensive data
off_train_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt",encoding='utf-8',sep="\t", header=None)
off_train_text.columns= ["tweet_text"]
off_train_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt",encoding='utf-8',sep="\t", header=None)
off_train_label.columns= ["tweet_label"]


off_val_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt",encoding='utf-8',sep="\t", header=None)
off_val_text.columns= ["tweet_text"]
off_val_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt",encoding='utf-8',sep="\t", header=None)
off_val_label.columns= ["tweet_label"]

off_test_text = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt",encoding='utf-8',sep="\t", header=None)
off_test_text.columns= ["tweet_text"]
off_test_label = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt",encoding='utf-8',sep="\t", header=None)
off_test_label.columns= ["tweet_label"]


off_map = pd.read_csv("https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/mapping.txt",encoding='utf-8',sep="\t", header=None)
off_map.columns= ["tweet_label","offensive"]

# Cleaning methods used by all datasets

In [ ]:
def clean_tweet(text):
  # "- Removing HTML tags
  #   - Removing punctuation
  #   - Lowering text
  #   "
  # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # convert text to lowercase
    text = text.strip().lower()
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    return text




In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)



In [ ]:

def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]



# **Emotion** dataset

## *Preprocessing*


In [ ]:
#concatenating dataframe by columns to make one dataset each for train, validation and test df
df_emo_train = pd.concat([emo_train_text,emo_train_label], axis=1)
df_emo_val = pd.concat([emo_val_text,emo_val_label], axis=1)
df_emo_test = pd.concat([emo_test_text,emo_test_label], axis=1)

In [ ]:
#joining mapping dataset with train, validation and test datasets
emo_combo_train = pd.merge(df_emo_train,emo_map, on='tweet_label')
emo_combo_val = pd.merge(df_emo_val,emo_map, on='tweet_label')
emo_combo_test = pd.merge(df_emo_test,emo_map, on='tweet_label')

In [ ]:
# concatenating train and validation datasets by row to make one complete training data
emo_train_val = pd.concat([emo_combo_train,emo_combo_val])

In [ ]:
train_emo = emo_train_val.copy()
train_emo['text_clean'] = train_emo['tweet_text'].apply(str).apply(lambda x: preprocess(x)) # text



## *Visualization*

In [ ]:
#getting top 20 words in each classification of Emotion
top_sad_words = get_top_n_words(emo_sad)
top_angry_words = get_top_n_words(emo_anger)
top_joy_words = get_top_n_words(emo_joy)
top_op_words = get_top_n_words(emo_opt)

s1 = [x[0] for x in top_sad_words[:20]]
s2 = [x[1] for x in top_sad_words[:20]]

a1 = [x[0] for x in top_angry_words[:20]]
a2 = [x[1] for x in top_angry_words[:20]]

j1 = [x[0] for x in top_joy_words[:20]]
j2 = [x[1] for x in top_joy_words[:20]]

o1 = [x[0] for x in top_op_words[:20]]
o2 = [x[1] for x in top_op_words[:20]]


## *Training* the model and testing with macro averaged F1 score as evaluation metric.

**cleaning** the data and then using Linear SVM for inital training and testing




Using BOW vectorizer from Python with countvectorizer from scikitlearn

In [ ]:
vectorizer = CountVectorizer(stop_words="english",preprocessor=clean_tweet)

training_features = vectorizer.fit_transform(emo_train_val["tweet_text"]) 
test_features = vectorizer.transform(emo_combo_test["tweet_text"])
model = LinearSVC()
model.fit(training_features,emo_train_val["emotion"])
y_pred = model.predict(test_features)

score = f1_score(emo_combo_test["emotion"], y_pred, average='macro')
print("F1 Score on emotion dataset: {}".format(round(score*100,2)))

F1 Score on emotion dataset: 61.03


Taking n-gram: 

In [ ]:
# modeling with countvectorizer and bigrams
vectorizer = CountVectorizer(stop_words="english",
                             preprocessor=clean_tweet,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(emo_train_val["tweet_text"])    
test_features = vectorizer.transform(emo_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, emo_train_val["emotion"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(emo_combo_test["emotion"], y_pred,average='macro')

print("F1 score on the emotion dataset: {:.2f}".format(score*100))


F1 score on the emotion dataset: 60.60


bigram also gave same result. next checking TF-IDF  vectorizer  with bigram

In [ ]:

# modeling with TF-IDF and bigrams
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_tweet,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(emo_train_val["tweet_text"])    
test_features = vectorizer.transform(emo_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, emo_train_val["emotion"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(emo_combo_test["emotion"], y_pred,average='macro')

print("F1 score on the emotion dataset: {:.2f}".format(score*100))


F1 score on the emotion dataset: 63.25


NOT  huge difference. Now trying the cleaning process given by tweeteval repository

In [ ]:
# cleaning with process suggested by tweeteval  and then vectorizing with TF-IDF and bigrams to train emotion dataset
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=preprocess,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(emo_train_val["tweet_text"])    
test_features = vectorizer.transform(emo_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, emo_train_val["emotion"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(emo_combo_test["emotion"], y_pred,average='macro')

print("F1 score Emotion dataset: {:.2f}".format(score*100))

F1 score Emotion dataset: 61.35


In [ ]:
#cleaning with process suggested by tweeteval  and then vectorizing with countvectorizer with bigrams to train emotion dataset by SVM

vectorizer = CountVectorizer(#stop_words="english",
                             preprocessor=preprocess,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(emo_train_val["tweet_text"])    
test_features = vectorizer.transform(emo_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, emo_train_val["emotion"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(emo_combo_test["emotion"], y_pred,average='macro')

print("F1 score on the emotion dataset: {:.2f}".format(score*100))

F1 score on the emotion dataset: 60.08


# **Hate Speech** dataset

## *Preprocessing*


In [ ]:
#concatenating dataframe to make one dataset each for train, validation and test df
df_hate_train = pd.concat([hate_train_text,hate_train_label], axis=1)
df_hate_val = pd.concat([hate_val_text,hate_val_label], axis=1)
df_hate_test = pd.concat([hate_test_text,hate_test_label], axis=1)

In [ ]:
hate_combo_train = pd.merge(df_hate_train,hate_map, on='tweet_label')
hate_combo_val = pd.merge(df_hate_val,hate_map, on='tweet_label')
hate_combo_test = pd.merge(df_hate_test,hate_map, on='tweet_label')

In [ ]:
hate_train_val = pd.concat([hate_combo_train, hate_combo_val])

In [ ]:
#checking total nan values/ empty rows
total =0
for n in pd.isna(hate_train_val["tweet_text"]):
  if n==True:
    total = total+1
total

8

In [ ]:
hate_train_val.dropna(inplace=True) #dropping the empty tweets, total 8, as they were included during the reading part
hate_combo_test.dropna(inplace=True)

In [ ]:
train_hate = hate_train_val.copy()
train_hate['text_clean'] = train_hate['tweet_text'].apply(str).apply(lambda x: preprocess(x))

In [ ]:
#removing workds like user and amp in clean tweets
train_hate['text_clean'] = train_hate['text_clean'].str.replace('user','')
train_hate['text_clean'] = train_hate['text_clean'].str.replace('amp','')
train_hate['text_clean'] = train_hate['text_clean'].str.replace('@','')
train_hate

,tweet_text,tweet_label,isHate,text_clean
0,@user nice new signage. Are you not concerned ...,0,not-hate,nice new signage. Are you not concerned by Be...
1,Hysterical woman like @user,0,not-hate,Hysterical woman like
2,Me flirting- So tell me about your father...,0,not-hate,Me flirting- So tell me about your father...
3,The Philippine Catholic bishops' work for migr...,0,not-hate,The Philippine Catholic bishops' work for migr...
4,When cuffin season is finally over,0,not-hate,When cuffin season is finally over
...,...,...,...,...
995,"shut the fuck up you know youre that bitch, wh...",1,hate,"shut the fuck up you know youre that bitch, wh..."
996,Where the fuck is your face scary hoe? Dont be...,1,hate,Where the fuck is your face scary hoe? Dont be...
997,Pass #MeritBased Immigration. Kill #ChainMigra...,1,hate,Pass #MeritBased Immigration. Kill #ChainMigra...
998,I usually dont hate people but I actually hate...,1,hate,I usually dont hate people but I actually hate...


In [ ]:
# Hate tweet
print("Hate Tweet example :",hate_train_val[hate_train_val['isHate']=='hate']['tweet_text'].values[0])
#Not-Hate Tweet 
print("Not Hate Tweet example :",hate_train_val[hate_train_val['isHate']=='not-hate']['tweet_text'].values[0])


Hate Tweet example : A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎 
Not Hate Tweet example : @user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you… 


## *Visualization*

In [ ]:
#creating different datasets for different emotions
hate_yes = train_hate[train_hate['isHate'] == 'hate']['text_clean']
hate_no = train_hate[train_hate['isHate'] == 'not-hate']['text_clean']


In [ ]:
#getting top 20 words in each classification of Hate dataset
top_hate_words = get_top_n_words(hate_yes)
top_nonHate_words = get_top_n_words(hate_no)


s1 = [x[0] for x in top_hate_words[:20]]
s2 = [x[1] for x in top_hate_words[:20]]

a1 = [x[0] for x in top_nonHate_words[:20]]
a2 = [x[1] for x in top_nonHate_words[:20]]



## *Training* the model and testing with macro averaged F1 score as evaluation metric.

In [ ]:
vectorizer = CountVectorizer(stop_words="english",preprocessor=clean_tweet)
training_features = vectorizer.fit_transform(hate_train_val["tweet_text"])  # feature extraction of train and validation set
test_features = vectorizer.transform(hate_combo_test["tweet_text"]) #feature extraction of test set
model = LinearSVC()
model.fit(training_features,hate_train_val["isHate"])
y_pred = model.predict(test_features)

score = f1_score(hate_combo_test["isHate"], y_pred, average='macro')
print("F1 Score on hate detection dataset: {}".format(round(score*100,2)))


F1 Score on hate detection dataset: 48.83


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# cleaning with basic clean funtion  and then vectorizing with TF-IDF and bigrams for Hate dataset
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_tweet,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(hate_train_val["tweet_text"])    
test_features = vectorizer.transform(hate_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, hate_train_val["isHate"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(hate_combo_test["isHate"], y_pred,average='macro')

print("F1 score on the Hate dataset: {:.2f}".format(score*100))

F1 score on the Hate dataset: 39.13


In [ ]:
# cleaning with proecss suggested by tweeteval  and then vectorizing with TF-IDF and bigrams hate dataset
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=preprocess,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(hate_train_val["tweet_text"])    
test_features = vectorizer.transform(hate_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, hate_train_val["isHate"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(hate_combo_test["isHate"], y_pred,average='macro')

print("F1 score on the Hate detection dataset: {:.2f}".format(score*100))

F1 score on the Hate detection dataset: 41.38


In [ ]:
# cleaning with proecss suggested by tweeteval  and then vectorizing with countvectorizer and bigrams hate dataset
vectorizer = CountVectorizer(#stop_words="english",
                             preprocessor=preprocess,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(hate_train_val["tweet_text"])    
test_features = vectorizer.transform(hate_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, hate_train_val["isHate"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(hate_combo_test["isHate"], y_pred,average='macro')

print("F1 score on the Hate detection dataset: {:.2f}".format(score*100))

F1 score on the Hate detection dataset: 48.65


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Best score is generated with countvectorizer, basic cleaning and wothout bigrams



# **Offensive** dataset 

## *Preprocessing*


In [ ]:
#concatenating dataframe to make one dataset each for train, validation and test df
df_off_train = pd.concat([off_train_text,off_train_label], axis=1)
df_off_val = pd.concat([off_val_text,off_val_label], axis=1)
df_off_test = pd.concat([off_test_text,off_test_label], axis=1)

In [ ]:
off_combo_train = pd.merge(df_off_train,off_map, on='tweet_label')
off_combo_val = pd.merge(df_off_val,off_map, on='tweet_label')
off_combo_test = pd.merge(df_off_test,off_map, on='tweet_label')

In [ ]:
off_train_val = pd.concat([off_combo_train,off_combo_val])

In [ ]:
train_off = off_train_val.copy()
train_off['text_clean'] = train_off['tweet_text'].apply(str).apply(lambda x: preprocess(x))

In [ ]:
#removing workds like user and amp in clean tweets
train_off['text_clean'] = train_off['text_clean'].str.replace('user','')
train_off['text_clean'] = train_off['text_clean'].str.replace('amp','')
train_off['text_clean'] = train_off['text_clean'].str.replace('@','')

train_off

,tweet_text,tweet_label,offensive,text_clean
0,@user Bono... who cares. Soon people will unde...,0,not-offensive,Bono... who cares. Soon people will understan...
1,@user Get him some line help. He is gonna be j...,0,not-offensive,Get him some line help. He is gonna be just f...
2,@user @user She is great. Hi Fiona!,0,not-offensive,She is great. Hi Fiona!
3,@user @user @user @user @user @user @user @use...,0,not-offensive,"This is the VetsResistSquadron""..."
4,@user @user Lol. Except he’s the most successf...,0,not-offensive,Lol. Except he’s the most successful preside...
...,...,...,...,...
1319,@user @user @user Weak argument considering -...,1,offensive,Weak argument considering -GOP congressmen...
1320,@user Stop saying this shit you are not going ...,1,offensive,Stop saying this shit you are not going to do...
1321,@user @user @user @user @user Sorry to break i...,1,offensive,Sorry to break it to you but the god of t...
1322,. she. is the most powerful woman on this eart...,1,offensive,. she. is the most powerful woman on this eart...


## *Training* the model and testing with macro averaged F1 score as evaluation metric.

In [ ]:
vectorizer = CountVectorizer(stop_words="english",preprocessor=clean_tweet)

training_features = vectorizer.fit_transform(off_train_val["tweet_text"])  # feature extraction of train and validation set
test_features = vectorizer.transform(off_combo_test["tweet_text"]) #feature extraction of test set

In [ ]:

model = LinearSVC()
model.fit(training_features,off_train_val["offensive"])
y_pred = model.predict(test_features)

score = f1_score(off_combo_test["offensive"], y_pred, average='macro')
print("F1 Score on offensive language dataset: {}".format(round(score*100,2)))

F1 Score on offensive language dataset: 71.48


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# cleaning with basic clean funtion  and then vectorizing with TF-IDF and bigrams for Offensive dataset
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_tweet,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(off_train_val["tweet_text"])    
test_features = vectorizer.transform(off_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, off_train_val["offensive"])
y_pred = model.predict(test_features)

# Evaluation 
score = f1_score(off_combo_test["offensive"], y_pred,average='macro')

print("F1 score on the offensive language dataset: {:.2f}".format(score*100))

F1 score on the offensive language dataset: 74.42


In [ ]:
# cleaning with  clean funtion given by tweeteval  and then vectorizing with countvectorizer and bigrams for Offensive dataset
vectorizer = CountVectorizer(stop_words="english",
                             preprocessor=preprocess,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(off_train_val["tweet_text"])    
test_features = vectorizer.transform(off_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, off_train_val["offensive"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(off_combo_test["offensive"], y_pred,average='macro')

print("F1 score on the offensive language dataset: {:.2f}".format(score*100))

F1 score on the offensive language dataset: 71.84


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# cleaning with clean funtion given by tweeteval  and then vectorizing with TF-IDF and bigrams for Offensive dataset
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=preprocess,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(off_train_val["tweet_text"])    
test_features = vectorizer.transform(off_combo_test["tweet_text"])

# Training
model = LinearSVC()
model.fit(training_features, off_train_val["offensive"])
y_pred = model.predict(test_features)

# Evaluation
score = f1_score(off_combo_test["offensive"], y_pred,average='macro')

print("F1 score on the offensive language dataset: {:.2f}".format(score*100))

F1 score on the offensive language dataset: 73.17
